In [1]:
import os
import numpy as np
import pandas as pd
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample as sobol_sample
from SALib.analyze.sobol import analyze as sobol_analyze
import matplotlib.pyplot as plt
from autoemulate.compare import AutoEmulate
from SALib.analyze.sobol import analyze
import numpy as np
import json
from SALib.sample import saltelli
from SALib.analyze import sobol

In [2]:
# Load them back
n_samples = 256
simulation_out_path = f'outputs/simulations/output_{n_samples}_samples'

with open(os.path.join(simulation_out_path,"bool_indices.json"), 'r') as f:
    bool_indices = json.load(f)

- load input_256_samples.csv
- load simulation_summary.csv 

Get rid of those rows in input_256_camples that correspond to bool_indices

In [3]:
# Load input_256_samples.csv
input_csv = os.path.join(simulation_out_path, "input_samples_256.csv")
X = pd.read_csv(input_csv)
print("Loaded input samples:", X.shape)

# Load simulation_summary.csv
summary_csv = os.path.join(simulation_out_path, "simulations_summary.csv")
Y = pd.read_csv(summary_csv)
print("Loaded simulation summary:", Y.shape)

Loaded input samples: (256, 35)
Loaded simulation summary: (255, 32)


In [4]:
# Lets get cleaning and removing 
# Remove rows at the indices in bool_indices
X_cleaned = X.drop(index=bool_indices).reset_index(drop=True)
print("Cleaned input samples:", X_cleaned.shape)  # Should be (255, 35)

Cleaned input samples: (255, 35)


In [5]:
# Generate random 128 indices between 0-255
random_indices = np.random.choice(X_cleaned.index, size=128, replace=False)

# Select the rows at these indices
X_random = X_cleaned.loc[random_indices].reset_index(drop=True)

# do the same for Y
Y_random = Y.loc[random_indices].reset_index(drop=True)

In [6]:
parameter_names = list(X.columns)
parameter_bounds = []
for name in parameter_names:
    min_val = X_random[name].min()
    max_val = X_random[name].max()
    parameter_bounds.append([min_val, max_val])

problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds
}
print(problem)



{'num_vars': 35, 'names': ['ao.r', 'ao.c', 'art.r', 'art.c', 'ven.r', 'ven.c', 'av.r', 'mv.r', 'la.E_pas', 'la.E_act', 'la.v_ref', 'la.k_pas', 'lv.E_pas', 'lv.E_act', 'lv.v_ref', 'lv.k_pas', 'T', 'ao.l', 'ao.v_ref', 'art.l', 'art.v_ref', 'ven.l', 'ven.v_ref', 'la.v', 'la.delay', 'la.t_tr', 'la.tau', 'la.t_max', 'lv.delay', 'lv.t_tr', 'lv.tau', 'lv.t_max', 'ao.v', 'art.v', 'ven.v'], 'bounds': [[120.4730413109064, 356.97165973484516], [0.1501288413070142, 0.4497803824022411], [566.5598240448162, 1680.8077952591702], [1.5169116714969275, 4.496986096724868], [4.507128020748496, 13.392014248296618], [67.19221204295755, 199.50710692759608], [3.0191556215286255, 8.935366937890649], [2.060695794876665, 6.149927610345184], [0.2251653180643916, 0.6590420363843441], [0.2264461976476013, 0.673807582911104], [5.00167858786881, 14.911303641274571], [0.0170368927636183, 0.0747955199363641], [0.5108328396454453, 1.496269602328539], [1.511625526472926, 4.49680962972343], [5.069448268041015, 14.98413668

In [7]:
import numpy as np

output_feature = "v_la_mean"  # Change to any column name in Y

Y_feature = Y_random[output_feature].values  # .values converts to numpy array.

# Run Sobol sensitivity analysis (first-order only)
sobol_indices = sobol_analyze(problem, Y_feature, calc_second_order=False)

# Sorting using argsort function S1 indices returns in ascending order, so we reverse it with [::-1], so biggest first
names = np.array(problem['names'])  # Names should match columns of X_cleaned
S1 = np.array(sobol_indices['S1'])
S1_conf = np.array(sobol_indices['S1_conf'])  # Uncertainties of the S1 indices
sorted_idx = np.argsort(S1)[::-1]
S1_sorted = S1[sorted_idx]
S1_conf_sorted = S1_conf[sorted_idx]
names_sorted = names[sorted_idx]

# Show the sorted sensitivities
for n, s, c in zip(names_sorted, S1_sorted, S1_conf_sorted):
    print(f"{n}: S1={s:.3f} ± {c:.3f}")

/Users/syonis/project/comparative-gsa/venv/lib/python3.11/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


RuntimeError: 
        Incorrect number of samples in model output file.
        Confirm that calc_second_order matches option used during sampling.